# Artificial Neural Networks - II
[COMP20121 Machine Learning for Data Analytics](https://sites.google.com/site/hejunhomepage/Teaching/machine-learning-for-data-analytics)

Author: Jun He

## Learning Objectives
1. What are  hyper-parameters in MLP and how to tune them with Sklearn? 
1. What is backpropagation for training MLP? 

## Part 1 Hyper-parameters of MLP and Tuning with Sklearn


### Multilayer perceptron (MLP)
* A multilayer perceptron (MLP) is one of most common artificial neural networks  (ANNs)  
* It consists of a layered, feedforward, completely connected network of artificial neurons (nodes)
    * Input layer: three nodes holding three features in a data set
    * Hidden layer: linked to other layers with weights
    * Output layer: one node holding the target variable  

 
<img src="https://i.ibb.co/vw8GWsy/MLP.jpg" width="500"> 

 

### Hyperparameters 
* Number of hidden layers
* Number of neurons in each hidden layer 
 
|<img src="https://qph.fs.quoracdn.net/main-qimg-65fa680a5effca84096237df3eb4ae88" width =400>|
|:--:|
|[One hidden layer MLP and Three layer MLP](https://missinglink.ai/wp-content/uploads/2018/11/networkgraph-1.png)|

* In `sklearn.neural_network.MLPClassifier`
    * hidden_layer_sizes = (9): one hidden layer with 9 neurons
    * hidden_layer_sizes = (9, 9, 9): three hidden layers with 9,9 and 9 neurons respectively
    * default (100,):  one hidden layer with 100 neurons

`clf = MLPClassifier(hidden_layer_sizes=(9, 9,9))`

### Example: Tune the number of hidden layers and neurons 
* Sklearn provides two generic approaches to hyperparameter tuning
    * `GridSearchCV`: exhaustively considers all parameter combinations 
    * `RandomizedSearchCV`: sample a given number of candidates from all parameter combinations with a specified distribution. 
* In the example, we apply `GridSearchCV` to MLPClassifier on the digits dataset   

### Data and Task 
* Data: UCI " Adult Census Income" dataset https://www.kaggle.com/uciml/adult-census-income
* Task: build a MLP model for predicting whether income exceeds 50K yr based on the census data 
* Features: 12
* Target:  `income` with two values `<=50K` and `>50K`
* Number of samples: 32561

In [1]:
import pandas as pd    
df = pd.read_csv('../input/adult-census-income/adult.csv' )
df.head() 

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


### Create target variable
* create the target variable `y` using a lambda function
    * if `income>50K`, `y=1`
    * if `incomde<=50K`, `y=0`

In [2]:
low = '<=50K' 
y = df['income'].apply(lambda x:0 if x==low else 1) 
y.head(5)

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

### Generate feature sets
* drop target `income` column from the dataframe

In [3]:
X = df.drop(['income'],axis=1)
X.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


### One-Hot Encode all categorical features 
* For the sake of presentation, we will not handle missing values and do not apply OrdinalEncoder to any ordinal data.
* Use  `pandas.get_dummies` to encode categorical features  
* `pandas.get_dummies` treats missing values (`?` in this data set) as an extra column (`workclass_?`)

In [4]:
X = pd.get_dummies(X) 
X.head(5)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,90,77053,9,0,4356,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,82,132870,9,0,4356,18,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,66,186061,10,0,4356,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,54,140359,4,0,3900,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,41,264663,10,0,3900,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Prepare Data
* Split data into Training and Test Data
* Data normalization by `MinMaxScaler`

In [5]:

from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split data into training and test data. Improvement: usually need validation data for hyperparameter tuning
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)# Fit only to the training data
# Now apply the transformations to both training and test data
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

### Build MLP model 
* Which model is better? 
    * MLP (3): one hidden layer with 3 nodes
    * MLP (3,3): two hidden layers with 3 and 3 nodes respectively
* Set the parameter grid 
    * `'hidden_layer_sizes': [(3), (3,3)]` 

In [6]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1,  max_iter=3000)  
parameter_grid = {
    'hidden_layer_sizes': [(3),   (3,3)], 
}

### Prepare grid search
* Import `GridSearchCV`
* Create a `GridSearchCV` object  where `cv=5` means 5-fold cross validation,
* Fit the `GridSearchCV` object  on training data

In [7]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(clf, parameter_grid,  cv=5)
gs = gs.fit(X_train,y_train)

### Search the best model and Train the best model
* Select the best classifier found in the Grid Search 
* Fit the best classifier on training data

In [8]:
#set the clf to the best combination of parameters
clf_best = gs.best_estimator_
print("best model:", clf_best.get_params())
# Fit the best model to the data. 
clf_best = clf_best.fit(X_train, y_train)

best model: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 3, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 3000, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


### Evaluate the best model
* Predict the model output on test data
* Evaluate the model accuracy

In [9]:
from sklearn import metrics
y_pred = clf_best.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.8587368205548163


## Part 2 MLP Training (1):  Feedforward for Calculating Output and Error 

### Prediction error 

* The **prediction error** on an output node is the difference between the actual and predicted output
\begin{align*}
    \mbox{SSE} (\mbox{one output node i})&= (\mbox{actual value}- \mbox{predicted value})^2\\
    SSE_i&= (y_i- \hat{y}_i)^2  
    \end{align*}
    * index $i$ for node $i$
    * $\hat{y}$ is the predicted value
    * $y$ the actual value 
* The sum of squared errors (SSE)  
     \begin{align*}
     \mbox{SSE} &= \sum_{\mbox{training samples}} \,\sum_{\mbox{output nodes}} (\mbox{actual value}- \mbox{predicted value})^2\\
    \mbox{SSE}&= \sum_k \sum_i(y_{ki}- \hat{y}_{ki})^2
    \end{align*}
    * index $k$ for summation over training samples
    * index $i$ for summation over ouput  nodes
* The mean of squared errors (MSE) 
$$
MSE=\frac{SSE}{\mbox{number of training samples}}
$$ 
* Prediction error is often called **loss function** or **cost function** in machine learning

### Example: Calculate Output and Error
* MLP   
<img src="https://i.ibb.co/vw8GWsy/MLP.jpg" width="500"> 

* Data inputs and  Initial Weights

| input    | weights   | weights   | weights   |
|----------|-----------|-----------|-----------|
|          | W0A = 0.5 | W0B = 0.7 | W0Z = 0.5  |
| x1 = 0.4 | W1A = 0.6 | W1B = 0.9 | WAZ = 0.9 |
| x2 = 0.2 | W2A = 0.8 | W2B = 0.8 | WBZ = 0.9 |
| x3 = 0.7 | W3A = 0.6 | W3B = 0.4 |           |
 

* Actual output: 0.8 


### Example: calculate output and prediction error
*  Summation on node A 

\begin{align*}
    \sum_A &=W_{0A} + W_{1A} x_{1} + W_{2A} x_{2} + W_{3A} x_{3} \\
    & = 0.5+0.6 \times 0.4 + 0.8 \times 0.2 + 0.6 \times 0.7 =1.32
\end{align*}

* Activiation on node $A$ by logistic activation function

\begin{align*}
 f(\sum_A) = \frac{1}{1 + \exp(–\sum_A)} =\frac{1}{1 + \exp(–1.32)} = 0.7892.
\end{align*}

* The output of node $A$: $Output_A =0.7892$
* Do summation and activation over all nodes

| node | summation $\sum$ | activation $f(\sum)$ |
|------|------------------|----------------|
| A    | 1.32             | 0.7892         |
| B    | 1.5              | 0.8176         |
| Z    | 1.946            | 0.8750         |

* Output on output node Z: 0.8750 
*  SSE on output Node Z 
\begin{align*}
    \mbox{SSE} (\mbox{Node Z})&= (\mbox{actual value}- \mbox{predicted value})^2 \\ &=(0.8-0.875)^2 = 0.005625
\end{align*}
 
 


## Part 3  MLP Training (2): Backpropagation for Optimizing Weights 
### Prediction Error Depends on Weights
* Given two models, which model is better? 
* MLP 1 with Weights 
    
| input    | weights   | weights   | weights   | SSE |
|----------|-----------|-----------|-----------|-----------|
|          | W0A = 0.5  | W0B = 0.7  | W0Z = 0.5  ||
| x1 = 0.4 | W1A = 0.6 | W1B = 0.9 | WAZ = 0.9 ||
| x2 = 0.2 | W2A = 0.8 | W2B = 0.8 | WBZ = 0.9 ||
| x3 = 0.7 | W3A = 0.6 | W3B = 0.4 |  |0.005625|

*  MLP 2 with all weights =0. In this case, $Output_Z =1/(1+\exp(0))=0.5$ (i.e., random guess in binary classification). $Error = 0.8-0.5=0.3$.  

| input    | weights   | weights   | weights   | SSE |
|----------|-----------|-----------|-----------|-----------|
|          | W0A = 0  | W0B = 0  | W0Z = 0  ||
| x1 = 0.4 | W1A = 0 | W1B = 0 | WAZ = 0 ||
| x2 = 0.2 | W2A = 0 | W2B = 0 | WBZ = 0 ||
| x3 = 0.7 | W3A = 0 | W3B = 0 |  |0.09|

* Question: how to find the weights of the optimal model? 



### Training  = Optimization 
|<img src = "https://i.ibb.co/8sz9G3y/ANN-optimization1.png" width =300>|
|:--:|
|$error (W) = W_1^2 + W_2^2$ where the optimal weights $W_1=W_2=0$ and $error =0$|

* Given data samples and actual output on training data, prediction error is an function of weights: $error(W) $
* MLP training is to find the optimal weights $W$ which minimizes the prediction error
    $$\mbox{minimize } error(W)$$  
* Optimization =  searching the lowest point on a landscape
    * error =  the height 
    * weights = coordinate axes 
    * the minimization problem intuitively can be explained as finding the position where  the height is the lowest  
    


### Search Algorithms 
|<img src="https://i.ibb.co/DVVsBjJ/ANN-optimization2.png" width=300>|
|:--:|
|Search the minima of $error(W) = W_1^2 + W_2^2$ from initial weights $W_1=W_2=9$|

* A search algorithm is an iterative process to find the minima
    1. Initialize a position $\mathbf{w}_{init}$
    1. WHILE (stopping criterion is not satisfied)
        1. Start at current position (weights $\mathbf{w}_{curent}$)
        1. Find a downhill search direction $\Delta \mathbf{w}_{current}$  
        1. Move along the downhill direction with a small step $\eta$ ($\eta$ is called learning rate in machine learning)
        1. Arrive at a new position (weights) $\mathbf{w}_{new} =\mathbf{w}_{current}+ \eta \Delta \mathbf{w}_{current}$ 
    1. ENDWHILE
* Steepest descent: downhill direction $\Delta \mathbf{w}_{current} = - \mbox{Gradient}$
 

### Local optimum, global optimum and convergence
  
|<img src ="https://miro.medium.com/max/1400/1*ZC9qItK9wI0F6BwSVYMQGg.png" width =400>|
|:--:|
|Starting from A, search might stop at local minima but not at global optima|

* A landscape may consist of local and global optima 
    * local optimum:  a position whose neighbour points have a higher height
    * global optimum: the position with the lowest height on the landscape
* A search algorithm is called local convergent   if it stops at a local optimum
* A search algorithm is called global convergent   if it stops at a global optimum
 

### Learning Rate 

<img src = "https://www.fromthegenesis.com/wp-content/uploads/2020/04/lr_12420_1-768x288.png" width =500>

* Learning rate $\eta (0 < \eta <1)$:  to control the search step size  
* What value should $\eta$ take?  
    * Too small: the weight adjustments tend to be very small. Thus, gradient descent converges to the global minimum very slow
    * Too large:  may fail to converge (red line)

### Backpropagation Algorithm 
|<img src ="https://i.ibb.co/8KP4ncB/MLP-forward-backward.jpg" width =500>|
|:--:|
|Feedforward and Backpropagation|

1. Initialize weights  
1. Read in samples and actual outputs
1. WHILE (stopping criterion is not satisfied)
    1. Compute  prediction output and prediction error by working **forward through the layers** (input -> hidden -> output)
    1. Adjust the weights by working **backward through the hidden layers**  (output -> hidden -> input)
1. ENDWHILE
 

### Termination Criteria 
* The training of ANNs is an iterative procedure
* What  serves as the stopping criterion?
    * simply set the maximal number of iterations 
        * `MLPClassifier(max_iter=300)` 
    * or stop when error change is too small, below a predefined threshold 
        * `MLPClassifier(tol=0.0001)` 
 

## Part 4 Example:  Backpropagation 
###  Backpropagation Rule 
|<img src ="https://i.ibb.co/8KP4ncB/MLP-forward-backward.jpg" width =500>|
|:--:|
|Backpropagation|
 
 

* The rule of updating  weights  (Mitchell, T. Machine Learning, 1997)  is based on stochastic gradient descent \begin{align}     W_{ij, new} &=W_{ij,current} +   \eta \delta_j x_{ij}      \end{align}
* $x_{ij}$ signifies the $i$th input to node $j$
* $\delta_j$ represents the error responsibilities of node $j$

\begin{align*}   \mbox{for an output layer node $j$:   }     \delta_j &= {output}_j (1-{output}_j) (actual_j -output_j), \\ \mbox{for a hidden layer node $j$:  } \delta_j &=  {output}_j (1-{output}_j) \displaystyle \sum_{k: \mbox{downstreamnodes}} W_{jk}\delta_k \end{align*}

 where $\sum_{k: \mbox{downstreamnodes}} $ refers to the weighted sum of the error responsibilities over downstream  nodes $k$
 
 * Hidden nodes have no actual value


### MLP and Output after first iteration
* Data inputs, Initial Weights, Output on Node Z and Error

| input    | weights   | weights   | weights   | Output | SSE|
|----------|-----------|-----------|-----------|-----------|-------|
|          | W0A = 0.5  | W0B = 0.7 | W0Z = 0.5  |||
| x1 = 0.4 | W1A = 0.6 | W1B = 0.9 | WAZ = 0.9 |||
| x2 = 0.2 | W2A = 0.8 | W2B = 0.8 | WBZ = 0.9 |||
| x3 = 0.7 | W3A = 0.6 | W3B = 0.4 |  | 0.8751|0.00564001|

* Set the learning rate $\eta=0.1$

### Backpropagation: Adjust $W_{0Z}$ (bias weight)
* Weight $W_{0Z}$  
    * Calculate the error responsibility $\delta Z$ for the output node $Z$   
    \begin{align*}
    \delta_Z &={output}_Z (1-{output}_Z) (actual_Z -output_Z)\\
    &=0.8751(1-0.8751)(0.8-0.8751)=-0.0082
    \end{align*}
    * Adjust the bias weight $W_{0Z}$ (with constant input =1) using the back-propagation rules as follows
    \begin{align*}
    \Delta W_{0Z} &= \eta \delta_Z   =0.1(-0.0082) = -0.0082\\
    W_{0Z,new} &= W_{0Z,current} +\Delta W_{0Z} =0.5 -0.00082= 0.49918
    \end{align*}

### Backpropagation: Adjust  $W_{AZ}$ (weight  to an output layer node)
* Weight $W_{AZ}$ where $Z$ is the output node
    * Calculate the error responsibility $\delta Z= -0.0082 $ for the output node $Z$   
    * Adjust the weight $W_{AZ}$  using the back-propagation rules
    \begin{align*}
    \Delta W_{AZ} &= \eta \delta_{Z} \cdot {output}_A =0.1(-0.0082) (-0.7892) = -0.000647 \\
    W_{AZ,new} &= W_{AZ,current} +\Delta W_{AZ} = 0.9 -0.000647= 0.899353
    \end{align*}  

### Backpropagation: Adjust  $W_{1A}$ (weight to a hidden layer node)
* Weight $W_{1A}$ where $A$ is a hidden layer node 
    * Calculate the error responsibility $\delta A$  for the hidden layer node $A$  
    \begin{align*}
        \delta_A &={output}_A (1-{output}_A)   W_{AZ} \delta_{Z}\\
        &=0.7892 (1-0.0.7892) (0.9)(-0.0082) =-0.00123
    \end{align*}
    * Adjust the weight $W_{1A}$  using the back-propagation rules
    \begin{align*}
    \Delta W_{1A} &= \eta \delta_{A}    x_1  =0.1(-0.00123) (-0.4) = -0.0000492 \\
    W_{1A,new} &= W_{1A,current} +\Delta W_{1A} = 0.6 -0.0000492 = 0.599508
    \end{align*} 

### Backpropagation: Adjust Rest of Weights
* Adjust all weights using the backpropagation rule 

| input    | weights   | weights   | weights   | Output | SSE|
|----------|-----------|-----------|-----------|-----------| ----|
|          | W0A = 0.499877 | W0B = 0.69989  | W0Z = 0.49918  |||
| x1 = 0.4 | W1A = 0.599508 | W1B = 0.899956 | WAZ = 0.899353 |||
| x2 = 0.2 | W2A = 0.7999754 | W2B = 0.799978 | WBZ = 0.89933|||
| x3 = 0.7 | W3A = 0.5999139 | W3B = 0.399923 |  | 0.8745|0.00555025| 

* Using new weights, calculate new output on Node Z: 0.8745 
* Calculate SSE under new weights
* SSE is reduced: $0.00555025 <  0.00564001$




 

## Part 6 More Hyperparameters in MLP
### Search Algorithms for Weight Optimization 
* Sklearn provide three algorithms (called solvers)  
    * `lbfgs`: an optimizer in the family of quasi-Newton methods.
    * `sgd`: stochastic gradient descent.
    * `adam`: a stochastic gradient-based optimizer  
    * Default solver: `adam` 
        * `clf = MLPClassifier(solver='adam')`  


### Activation functions
* Sklearn supports 4 activation functions
    * Let $x= \sum$ after summation
    * `identity`, simply returns $f(x) = x$
    * `logistic`, the logistic sigmoid function, returns $f(x) = 1 / (1 + \exp(-x))$
    * `tanh`, the hyperbolic tan function, returns $f(x) = \tanh(x)$
    * `relu`,   the rectified linear unit function, returns $f(x) = \max(0, x)$
    * default: `relu`
        * `clf = MLPClassifier(activation=  'logistic')`   
* In 1990s, the `logistic` function was the default activation function  
* In late 1990s to 2010s, the  `tanh` function was the default activation function  
* Now, the `relu` function becomes the default activation function  

### Learning Rate 
* Sklearn supports learning rates  when solver=`sgd`
    * `constant`: a constant,  given by `learning_rate_init=0.001`.
    * `invscaling`: gradually decrease 
    * `adaptive` 

## Summary
* MLP uses **back-propagation algorithms** to find the optimal weights which minimise the prediction error
* Tune hyperparameters through GridSearch 
